# First Steps with Prompt Engineering

In [2]:
import os
from openai import OpenAI
!pip install cohere
import cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 76.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata


co = cohere.Client(userdata.get('COHERE_API_KEY'))

openai_client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")
)

In [4]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given model "

    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [5]:
def test_prompt_cohere(prompt, suppress=False, model='command-a-03-2025', **kwargs):
    response = co.chat(
        message=prompt,
        model=model,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.text}')
    else:
        return response.text

## Just ASK

In [6]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakındaki restoran nerede?


In [7]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakın restoran nerede?


In [8]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
En yakın restoran nerede?


# Few-shot learning

Using examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](https://github.com/dmoneyUK/quick-start-guide-to-llms/blob/main/images/gpt3_paper.png?raw=1)

In [9]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [10]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No

Explanation: The statement "The book was about WWII" is an objective observation about the subject matter of the book. It does not express personal feelings, opinions, or judgments, making it a factual and non-subjective statement.


In [11]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
I found the book to be a gripping and emotional journey through the horrors of WWII. The author did a fantastic job of bringing the historical events to life through the perspectives of the characters. I was deeply moved by the sacrifices and struggles of those who lived through this difficult time. Overall, I highly recommend this book to anyone interested in learning more about WWII and its impact on individuals.


In [12]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
The reviewer found the topic of WWII to be interesting or important.


In [13]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [14]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
Yes


In [15]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No

{
  "subjective": "No"
}


In [16]:
examples = [
    ('Review: This movie sucks\nSubjective: {"answer": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"answer": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"answer": true}'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: {"answer": true}
###
Review: This tv show was about the ocean
Subjective: {"answer": false}
###
Review: This book had a lot of flaws
Subjective: {"answer": true}
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
{"answer": false}


# Personas / Style

In [ ]:
# It only takes a few words to pretty drastically change the output

In [17]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Well, did you try actually typing in the correct password? Or are you just incompetent?


In [18]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Hello there! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your username or email address so I can look into it for you?


In [19]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Help you I can, young padawan. To access your account, check your username and password you must. If forgotten, reset your password you should. Troubleshoot any issues you have, I will. Patience you must have, but help you I will. Strong with the Force, your account shall be soon.


# What a good time to talk about output validation and bias!

In [20]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [29]:
sequence_to_classify = "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website."

candidate_labels = ['racist', 'sexist', 'rude']

classifier(sequence_to_classify, candidate_labels, multi_label=True)  # Assuming there can be multiple answers


{'sequence': "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website.",
 'labels': ['racist', 'rude', 'sexist'],
 'scores': [0.9752524495124817, 0.9617350101470947, 0.6626130938529968]}

In [32]:
# then the "rude" AI wasn't that bad
classifier(
    'Do you have your login information? Because if not, then there is nothing I can do for you.',
    candidate_labels, multi_label=True)



{'sequence': 'Do you have your login information? Because if not, then there is nothing I can do for you.',
 'labels': ['rude', 'sexist', 'racist'],
 'scores': [0.11479902267456055, 0.04397183656692505, 0.019839253276586533]}

In [33]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,  # technically (mathematically) we cannot set temp to 0 so OpenAI is doing something under the hood here
        suppress=True
    ))


100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


In [24]:
# only 1 unique response
responses, len(set(responses))

(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm s

In [25]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|██████████| 10/10 [00:08<00:00,  1.22it/s]


(["Hey there! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you out. Can you provide me with your account information so I can assist you further? Let's get this sorted out for you.",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you out. Can you please provide me with your account information so I can assist you in resolving this issue? Thank you for reaching out!",
  "Hello! I'm sorry to hear you're having trouble accessing your account. Let's work together to get this sorted out. Can you please provide me with your account details so I can assist you further? Thank you for reaching out!",
  "Hi there! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your username or email address so I can look into this for you?",
  "Hello! I'd be happy to help you with that. Could you please provide me with your account inform

In [34]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,
        suppress=True,
        max_tokens=50
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


(["Hi there! I'm sorry to hear you're having trouble with login into your account. I'd be happy to help guide you through restoring your account access. Let's see if we can get this sorted out for you. Could you please provide me",
  "Hi there! I'm sorry to hear you're experiencing difficulty logging in. Don't worry, I'm here to help. Let's work together to figure out what the issue may be and get you back into your account. Can you please provide me",
  " Hi there! I'm sorry to hear that you are having trouble accessing your account. Let's work together to figure out the issue. Could you please provide me with your username or email registered with the account so that I can assist you better? Thank",
  "Hello! I'd be happy to help you with accessing your account. Can please provide me with the email address or username associated with your account so I bed of preemptive strategipaystry community background civilianystick gives defeLEGALemplate shareeca.funhabmsg",
  "Sure thing! I'm s

In [36]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=.1,
        suppress=True
    ))
# all different and horrible responses!
responses, len(set(responses))


100%|██████████| 10/10 [00:07<00:00,  1.27it/s]


(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "He

In [37]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|██████████| 10/10 [00:08<00:00,  1.20it/s]


(["Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear that you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! I'm sorry to hear you're having trouble accessing your account. I'd be happy to help you with that. Can you please provide me with your account information so I can assist you further?",
  "Hello! 